<a href="https://colab.research.google.com/github/salmacmpeg/Satelite-Image-semantic-Segmentation/blob/main/Preprocessing_of_Dstl_Semantic_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#This is code is for the preprocessing step for the DSTL dataset
#this works only on the multispectral RGB bands.
#............................................
#Authors: 
# salma abdel Monem Abdel Motaleb and AbdelRahman Sobeih
#..............................................
#imports
from collections import defaultdict
import csv
import sys
import cv2
from shapely.geometry import MultiPolygon, Polygon
import shapely.wkt
import shapely.affinity
import numpy as np
import tifffile as tiff
import matplotlib.pyplot as plt
import pandas as pd
import image_slicer

In [ ]:
#this cell is from https://www.kaggle.com/lopuhin/full-pipeline-demo-poly-pixels-ml-poly
def get_scalers(h,y_min):
    h_ = h * (h / (h + 1))
    return  h_ / y_min

In [ ]:
#this cell is from https://www.kaggle.com/lopuhin/full-pipeline-demo-poly-pixels-ml-poly

def mask_for_polygons(polygons,h,w,c):
    img_mask = np.zeros((h,w), np.uint8)
    if not polygons:
        return img_mask
    int_coords = lambda x: np.array(x).round().astype(np.int32)
    exteriors = [int_coords(poly.exterior.coords) for poly in polygons]
    interiors = [int_coords(pi.coords) for poly in polygons
                 for pi in poly.interiors]
    cv2.fillPoly(img_mask, exteriors, c)
    cv2.fillPoly(img_mask, interiors, 0)
    return img_mask

In [ ]:
grid_sizes=pd.read_csv('F:\\dstl-satellite\\grid_sizes\\grid_sizes.csv')
grid_sizes.rename( columns={'Unnamed: 0':'ImageId'}, inplace=True )

In [ ]:
pdf3=pd.read_csv('F:\\dstl-satellite\\train_wkt_v4\\train_wkt_v4.csv')

multipolygons=pdf3[(pdf3['MultipolygonWKT']!='MULTIPOLYGON EMPTY')]
multipolygons['train_poly'] = multipolygons.apply(lambda row:  shapely.wkt.loads(str(row['MultipolygonWKT'])) , axis = 1) 
multipolygons=pd.merge(multipolygons, grid_sizes, on='ImageId')   

In [ ]:
images=[]
unique_ig=multipolygons['ImageId'].unique()
for i in range (len(unique_ig)):
    im_rgb = tiff.imread('F:\\dstl-satellite\\three_band\\three_band/{}.tif'.format(unique_ig[i])).transpose([1, 2, 0])
    images.append(im_rgb)
    shapeH=(im_rgb.shape[:2][0])
    shapeW=(im_rgb.shape[:2][1])
    multipolygons.loc[(multipolygons['ImageId']==unique_ig[i]),'H_original']=shapeH
    multipolygons.loc[(multipolygons['ImageId']==unique_ig[i]),'W_original']=shapeW

In [ ]:
multipolygons['y_scaler']=get_scalers(multipolygons['H_original'],multipolygons['Ymin'])
multipolygons['x_scaler']=get_scalers(multipolygons['W_original'],multipolygons['Xmax'])

In [ ]:

multipolygons['train_poly_scaled']=""
for i in range(multipolygons.shape[0]):
    multipolygons['train_poly_scaled'].iloc[i]= shapely.affinity.scale(
    multipolygons['train_poly'].iloc[i], xfact=multipolygons['x_scaler'].iloc[i], yfact=multipolygons['y_scaler'].iloc[i], origin=(0, 0, 0))
    


In [ ]:
unique_ig=multipolygons['ImageId'].unique()
train_masks=[]
for i in range (len(unique_ig)):
    img_id=unique_ig[i]
    classes=multipolygons[multipolygons['ImageId']==img_id]['ClassType']
    mask1=[]
    mask2=[]
    
    for j in classes:
        sub=multipolygons[(multipolygons['ImageId']==img_id)& (multipolygons['ClassType']==j)]
        #print((sub.iloc[0]))
        mask2=mask_for_polygons(sub['train_poly_scaled'].iloc[0],int(sub['H_original'].iloc[0]),int(sub['W_original'].iloc[0]),int(j) )
        if len(mask1)<1:
            mask1=mask2
        else:
            mask1=np.maximum(mask1,mask2)
    train_masks.append(mask1)

In [ ]:
#free some memory
del multipolygons['MultipolygonWKT']
del multipolygons['train_poly']
del multipolygons['train_poly_scaled']


In [ ]:
#save the data
import pickle
with open('F:\\dstl-satellite\\t_images_4', 'wb') as f:
    pickle.dump(images, f)
with open('F:\\dstl-satellite\\t_masks_4', 'wb') as f:
    pickle.dump(train_masks, f)